In [1]:
import h5py
import healpy as hp
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
!ls -lsh /project/chihway/dhayaa/DECADE/Foreground_Masks/*

385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:07 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.05_Star5.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:04 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.1_Star5.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Dec  3 19:33 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.25_Star5.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:19 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star10.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:22 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star20.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:10 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star3.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:13 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star4.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:16 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star5.f

# fiducial catalog alone

In [11]:
import fitsio

fitsio.read('/project/chihway/dhayaa/DES_Catalogs/psf_y3a1-v29.fits').shape

(56707987,)

In [2]:
Badcolor_map = hp.read_map('/project2/kadrlica/chinyi/DELVE_DR3_1_bad_colour_mask.fits', dtype = int)

In [3]:
PATHS = [
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star5.fits',
         ]

NAMES = ['DR3_hyperparam',
         ]

for NAME, PATH in zip(NAMES, PATHS):
    
    GOLD_Foreground  = hp.read_map(PATH, dtype = int)
        
    path = '/scratch/midway3/dhayaa/SHEARTESTS/%s' % NAME
    os.system('mkdir %s' % path)
    
    with h5py.File('/project/chihway/data/decade/metacal_gold_combined_20240209.hdf', 'r') as f:
        
        Region_mask = np.invert(f['DEC'][:] > np.where(f['RA'][:] < 225, 30 - (30 - 12)/(225 - 200) * (f['RA'][:] - 200), 12.))
        M = ((f['mcal_s2n_noshear'][:] > 3) & 
             (f['mcal_T_ratio_noshear'][:] > 0.2) & (f['mcal_flags'][:] == 0) & Region_mask)

        pix_assign       = hp.ang2pix(hp.npix2nside(GOLD_Foreground.size), f['RA'][:], f['DEC'][:], lonlat = True)
        FLAGS_Foreground = GOLD_Foreground[pix_assign]
        FLAGS_BADCOLOR   = Badcolor_map[pix_assign]
        M = M & (FLAGS_Foreground ==  0) & (FLAGS_BADCOLOR == 0)
        np.save(path + '/shear_inds.npy', np.where(M)[0])


    with h5py.File('/project/chihway/dhayaa/DECADE/star_psf_shapecat_20231216.hdf5', 'r') as f:
        M = (f['FLUX_AUTO'][:] / f['FLUXERR_AUTO'][:] > 1)

        Region_mask = np.invert(f['dec'][:] > np.where(f['ra'][:] < 225, 30 - (30 - 13)/(225 - 200) * (f['ra'][:] - 200), 13.))
        
        pix_assign       = hp.ang2pix(hp.npix2nside(GOLD_Foreground.size), f['ra'][:], f['dec'][:], lonlat = True)
        FLAGS_Foreground = GOLD_Foreground[pix_assign]
        FLAGS_BADCOLOR   = Badcolor_map[pix_assign]
        M = M & (FLAGS_Foreground ==  0) & (FLAGS_BADCOLOR == 0)
        
        inds = np.where(M)[0]
        Max_size = 200_000_000
        if len(inds) > Max_size:
            inds = np.random.default_rng(42).choice(inds, size = Max_size, replace = False)
            print("CHANGING LENGTH %d ---> %d" % (np.sum(M), len(inds)))
        
        np.save(path + '/psf_inds.npy', inds)
        
        
    JOB_TXT = """#!/bin/bash
#SBATCH --job-name ShearTests_%(NAME)s
#SBATCH --partition=caslake
#SBATCH --account=pi-kadrlica
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=48
#SBATCH --time=30:00:00
#SBATCH --output=/home/dhayaa/DECADE/shearcat/shear_tests/runs/%(NAME)s.log
#SBATCH --mail-user=dhayaa@uchicago.edu
#SBATCH --mail-type=BEGIN,END

#X=${TMPDIR}
#echo $TMPDIR
if [ "$USER" == "dhayaa" ]
then
    conda activate /project/chihway/dhayaa/MyEnvs_Midway3/shear
fi


RUN_DIR=/home/dhayaa/DECADE/shearcat/shear_tests

python -u $RUN_DIR/ShearTestRunner.py --psf_cat "/project/chihway/dhayaa/DECADE/star_psf_shapecat_20231216.hdf5" \\
                                      --galaxy_cat "/project/chihway/data/decade/metacal_gold_combined_20240209.hdf" \\
                                      --psf_cat_inds "%(PSF_INDS)s" \\
                                      --galaxy_cat_inds "%(SHEAR_INDS)s" \\
                                      --output_path "%(OUTPUT)s" \\
                                      --sim_Cls_path "/project/chihway/dhayaa/DECADE/cosmosis/Lucas_files/Kappa_Cls.txt" \\
                                      --All

    """
    
    args = {'NAME' : NAME,
            'PSF_INDS' : path + '/psf_inds.npy',
            'SHEAR_INDS' : path + '/shear_inds.npy',
            'OUTPUT' : path}
    
    with open('runs/job_%s.sh' % NAME, 'w') as f:
        
        f.write(JOB_TXT % args)

    print("DONE WITH", NAME)

mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/DR3_hyperparam’: File exists
/scratch/local/jobs/19175856/ipykernel_915898/1034493550.py:29: RuntimeWarning: invalid value encountered in divide
  M = (f['FLUX_AUTO'][:] / f['FLUXERR_AUTO'][:] > 1)


CHANGING LENGTH 809078890 ---> 200000000
DONE WITH DR3_hyperparam


# Make job files

In [4]:
JOB_TXT = """#!/bin/bash
#SBATCH --job-name ShearTests_%(NAME)s
#SBATCH --partition=caslake
#SBATCH --account=pi-jfrieman
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=48
#SBATCH --time=36:00:00
#SBATCH --output=/home/dhayaa/DECADE/shearcat/shear_tests/runs/%(NAME)s.log
#SBATCH --mail-user=dhayaa@uchicago.edu
#SBATCH --mail-type=BEGIN,END

#X=${TMPDIR}
#echo $TMPDIR
if [ "$USER" == "dhayaa" ]
then
    conda activate /project/chihway/dhayaa/MyEnvs_Midway3/shear
fi


RUN_DIR=/home/dhayaa/DECADE/shearcat/shear_tests

python -u $RUN_DIR/ShearTestRunner.py --psf_cat "/project/chihway/dhayaa/DECADE/star_psf_shapecat_20231216.hdf5" \\
                                      --galaxy_cat "/project/chihway/data/decade/metacal_gold_combined_20240209.hdf" \\
                                      --psf_cat_inds "%(PSF_INDS)s" \\
                                      --galaxy_cat_inds "%(SHEAR_INDS)s" \\
                                      --output_path "%(OUTPUT)s" \\
                                      --sim_Cls_path "/project/chihway/dhayaa/DECADE/cosmosis/Lucas_files/Kappa_Cls.txt" \\
                                      --Star_SNR_min %(Star_SNR_min)d\\
                                      --Npatch %(Npatch)d\\
                                      --MapNSIDE_weightrands %(MapNSIDE_weightrands)d\\
                                      --All

        """

In [5]:
for SNR in [1, 20, 40, 60, 80, 100, 150]:

    NAME = 'STARSNR_%d' %SNR
    path = '/scratch/midway3/dhayaa/SHEARTESTS/%s' % NAME
    os.system('mkdir %s' % path)

    args = {'NAME' : NAME,
            'PSF_INDS' : '/scratch/midway3/dhayaa/SHEARTESTS/DR3_hyperparam/' + '/psf_inds.npy',
            'SHEAR_INDS' : '/scratch/midway3/dhayaa/SHEARTESTS//DR3_hyperparam/' + '/shear_inds.npy',
            'OUTPUT' : path,
            'Star_SNR_min' : SNR,
            'Npatch' : 100,
            'MapNSIDE_weightrands' : 256}

    with open('runs/job_%s.sh' % NAME, 'w') as f:

        f.write(JOB_TXT % args)

mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/STARSNR_1’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/STARSNR_20’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/STARSNR_40’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/STARSNR_60’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/STARSNR_80’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/STARSNR_100’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/STARSNR_150’: File exists


In [6]:
for Npatch in [100, 150, 200, 300]:

    NAME = 'NPATCH_%d' %Npatch
    path = '/scratch/midway3/dhayaa/SHEARTESTS/%s' % NAME
    os.system('mkdir %s' % path)

    args = {'NAME' : NAME,
            'PSF_INDS' : '/scratch/midway3/dhayaa/SHEARTESTS/DR3_hyperparam/' + '/psf_inds.npy',
            'SHEAR_INDS' : '/scratch/midway3/dhayaa/SHEARTESTS//DR3_hyperparam/' + '/shear_inds.npy',
            'OUTPUT' : path,
            'Star_SNR_min' : 80,
            'Npatch' : Npatch,
            'MapNSIDE_weightrands' : 256}

    with open('runs/job_%s.sh' % NAME, 'w') as f:

        f.write(JOB_TXT % args)

mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/NPATCH_100’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/NPATCH_150’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/NPATCH_200’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/NPATCH_300’: File exists


In [7]:
for NSIDE in [128, 256, 512, 1024]:

    NAME = 'MAPNSIDE_%d' % NSIDE
    path = '/scratch/midway3/dhayaa/SHEARTESTS/%s' % NAME
    os.system('mkdir %s' % path)

    args = {'NAME' : NAME,
            'PSF_INDS' : '/scratch/midway3/dhayaa/SHEARTESTS/DR3_hyperparam/' + '/psf_inds.npy',
            'SHEAR_INDS' : '/scratch/midway3/dhayaa/SHEARTESTS//DR3_hyperparam/' + '/shear_inds.npy',
            'OUTPUT' : path,
            'Star_SNR_min' : 80,
            'Npatch' : 100,
            'MapNSIDE_weightrands' : NSIDE}

    with open('runs/job_%s.sh' % NAME, 'w') as f:

        f.write(JOB_TXT % args)

mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/MAPNSIDE_128’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/MAPNSIDE_256’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/MAPNSIDE_512’: File exists
mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/MAPNSIDE_1024’: File exists
